In [2]:
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

In [3]:
df = pd.read_csv('/Users/darinaafanaseva/Desktop/DataScience/project/data/_data.csv')
pd.set_option('display.max_columns', None)

In [4]:
df['Parking']=df['Парковка'].apply(lambda x: 0 if pd.isna(x) else 1)
df.drop(columns=['Парковка'], inplace=True)


In [5]:
# Столбец "Дополнительно" разделен на 11 столбцов, в каждом из которых проставлены 0/1 (наличие в квартире)
df['Furniture_in_the_rooms']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Мебель в комнатах' in x else 0)
df['Kitchen_furniture']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Мебель на кухне' in x else 0)
df['Bath']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Ванна' in x else 0)
df['Washing_machine']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Стиральная машина' in x else 0)
df['Air_conditioner']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Кондиционер' in x else 0)
df['Dishwasher']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Посудомоечная машина' in x else 0)
df['TV']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Телевизор' in x else 0)
df['Internet']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Интернет' in x else 0)
df['Telephone']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Телефон' in x else 0)
df['Shower_cabin']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Душевая кабина' in x else 0)
df['Fridge']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Холодильник' in x else 0)
df.drop(columns=['Дополнительно'], inplace=True)

In [6]:
df[['Балкон1', 'Балкон2']] = df['Балкон'].str.split(', ', expand=True)
df['value1'] = df['Балкон1'].str.extract(r'(\d+)')
df['value2'] = df['Балкон2'].str.extract(r'(\d+)')
df['value1'] = pd.to_numeric(df['value1'])
df['value2'] = pd.to_numeric(df['value2'])
df['Baclony'] = df['value1'] + df['value2']
df['Baclony'] = df['Baclony'].fillna(df['value1'])
df['Baclony']=df['Baclony'].fillna(0)


In [7]:
df[['Совмещенный', 'Раздельный']] = df['Санузел'].str.split(', ', expand=True)
df['Совмещенный'] = df['Совмещенный'].str.extract(r'(\d+)')
df['Раздельный'] = df['Раздельный'].str.extract(r'(\d+)')
df['Combined'] = pd.to_numeric(df['Совмещенный']).fillna(0)
df['Separate'] = pd.to_numeric(df['Раздельный']).fillna(0)
df['Bathroom'] = df['Combined'] + df['Separate']

In [8]:
df[['Пасс', 'Груз']] = df['Лифт'].str.split(', ', expand=True)
df['Пасс'] = df['Пасс'].str.extract(r'(\d+)')
df['Груз'] = df['Груз'].str.extract(r'(\d+)')
df['Passenger'] = pd.to_numeric(df['Пасс']).fillna(0)
df['Cargo'] = pd.to_numeric(df['Груз']).fillna(0)
df['Elevator'] = df['Passenger'] + df['Cargo']

In [9]:
df['Square,m2'] = df['Площадь, м2'].astype(str).str.split('/').str[0].astype(float)

In [10]:
df['Rooms'] = df['Количество комнат'].astype(str).str.split(',').str[0]
df['Rooms'] = pd.to_numeric(df['Rooms'], errors='coerce')
df['Rooms'] = df['Rooms'].astype('Int64')

In [11]:
df['Square,m2'] = pd.to_numeric(df['Square,m2'], errors='coerce')
df_notna = df.dropna(subset=['Rooms', 'Square,m2'])
average_room_size = (df_notna['Square,m2'] / df_notna['Rooms']).mean()

In [12]:
df['Rooms'] = df.apply(
    lambda row: np.round(row['Square,m2'] / average_room_size) if pd.isnull(row['Rooms']) else row['Rooms'],
    axis=1
).astype('Int64')

In [13]:
df['Renovation'] = df['Ремонт'].fillna('Не указано')

In [14]:
df['Children'] = np.where(df['Можно с детьми/животными'].str.contains('Можно с детьми', na=False), 1, 0)
df['Animal'] = np.where(df['Можно с детьми/животными'].str.contains('Можно с животными', na=False), 1, 0)

In [15]:
mean_height = df['Высота потолков, м'].mean().round(1)
df['Height,m'] = df['Высота потолков, м'].fillna(mean_height)
df = df.rename(columns={
 'ID  объявления': 'ID',
 'Адрес': 'Address',  
})


In [16]:
df[['Floor', 'House']] = df['Дом'].str.split(',', expand=True)
df[['Floor_flat', 'Floor_all']] = df['Floor'].str.split('/', expand=True)
df['House'] = df['House'].fillna('Неизвестный')
df['Floor_flat'] = df['Floor_flat'].astype('Int64')
df['Floor_all'] = df['Floor_all'].astype('Int64')

In [17]:
price = df['Цена'].dropna()
rent_price = price.apply(lambda x: x.split('/')[0])
df['Price'] = rent_price.apply(lambda x: float(x.split(' ')[0])/1000)

In [18]:
pd.set_option('display.max_colwidth', None)
df['Цена']

0        500000.0 руб./ За месяц, Залог - 500000 руб., Коммунальные услуги включены, Срок аренды - Длительный, Предоплата 1 мес
1        500000.0 руб./ За месяц, Залог - 500000 руб., Коммунальные услуги включены, Срок аренды - Длительный, Предоплата 1 мес
2                                      500000.0 руб./ За месяц, Залог - 500000 руб., Срок аренды - Длительный, Предоплата 1 мес
3                                      400000.0 руб./ За месяц, Залог - 400000 руб., Срок аренды - Длительный, Предоплата 1 мес
4        225000.0 руб./ За месяц, Залог - 225000 руб., Коммунальные услуги включены, Срок аренды - Длительный, Предоплата 1 мес
                                                                  ...                                                          
23363      42000.0 руб./ За месяц, Залог - 42000 руб., Коммунальные услуги включены, Срок аренды - Длительный, Предоплата 1 мес
23364      45000.0 руб./ За месяц, Залог - 45000 руб., Коммунальные услуги включены, Срок аренды - Длите

In [19]:
df_1 = df[['ID', 'Rooms','Address','Square,m2','House','Floor_all','Floor_flat','Price',
           'Renovation','Baclony','Bathroom','Children','Animal','Height,m','Elevator',
           'Parking','Furniture_in_the_rooms','Kitchen_furniture','Bath','Washing_machine',
           'Air_conditioner','Dishwasher','TV','Internet','Telephone','Shower_cabin','Fridge']]
df_1

,ID,Rooms,Address,"Square,m2",House,Floor_all,Floor_flat,Price,Renovation,Baclony,Bathroom,Children,Animal,"Height,m",Elevator,Parking,Furniture_in_the_rooms,Kitchen_furniture,Bath,Washing_machine,Air_conditioner,Dishwasher,TV,Internet,Telephone,Shower_cabin,Fridge
0,271271157,4,"Москва, улица Новый Арбат, 27",200.0,Монолитный,16,5,500.0,Дизайнерский,0.0,0.0,1,1,3.00,5.0,1,1,1,1,1,1,1,1,1,1,1,1
1,271634126,4,"Москва, улица Новый Арбат, 27",198.0,Монолитно-кирпичный,16,5,500.0,Дизайнерский,0.0,3.0,1,0,3.50,2.0,1,1,1,1,1,1,1,1,1,0,1,1
2,271173086,4,"Москва, улица Новый Арбат, 27",200.0,Неизвестный,16,5,500.0,Евроремонт,0.0,3.0,1,0,3.20,1.0,1,1,1,1,1,1,1,1,1,1,1,1
3,272197456,4,"Москва, переулок Плотников, 21С1",170.0,Неизвестный,6,5,400.0,Евроремонт,0.0,3.0,0,1,3.20,1.0,1,1,1,1,1,1,1,1,1,1,1,1
4,273614615,2,"Москва, улица Новый Арбат, 15",58.0,Панельный,26,12,225.0,Евроремонт,0.0,2.0,0,0,3.90,2.0,0,1,1,1,1,0,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23363,215565511,1,"Москва, Боровское шоссе, 2к7, ш. Боровское (3 км до МКАД), ш. Сколковское (8 км до МКАД)",35.0,Неизвестный,14,10,42.0,Евроремонт,1.0,1.0,0,0,3.00,2.0,0,1,1,1,1,1,0,0,1,0,0,1
23364,274654844,1,"Москва, Производственная улица, 8к1, ш. Боровское (5 км до МКАД), ш. Киевское (7 км до МКАД)",38.7,Монолитный,18,5,45.0,Евроремонт,1.0,1.0,0,0,3.00,2.0,0,1,1,1,1,0,0,1,0,0,0,1
23365,268679909,2,"Москва, Боровский проезд, 11",43.1,Кирпичный,5,5,50.0,Дизайнерский,1.0,1.0,1,0,3.00,0.0,0,0,1,0,1,1,0,0,1,0,1,1
23366,274807525,2,"Москва, улица Богданова, 6к1, ш. Боровское (3 км до МКАД), ш. Сколковское (8 км до МКАД)",52.5,Монолитный,23,8,55.0,Евроремонт,1.0,2.0,0,0,2.65,3.0,1,1,1,1,1,1,1,0,0,0,0,1


In [20]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23368 entries, 0 to 23367
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      23368 non-null  int64  
 1   Rooms                   23368 non-null  Int64  
 2   Address                 23368 non-null  object 
 3   Square,m2               23368 non-null  float64
 4   House                   23368 non-null  object 
 5   Floor_all               23368 non-null  Int64  
 6   Floor_flat              23368 non-null  Int64  
 7   Price                   23368 non-null  float64
 8   Renovation              23368 non-null  object 
 9   Baclony                 23368 non-null  float64
 10  Bathroom                23368 non-null  float64
 11  Children                23368 non-null  int64  
 12  Animal                  23368 non-null  int64  
 13  Height,m                23368 non-null  float64
 14  Elevator                23368 non-null

In [21]:
df_1.to_csv('data.csv')